<b>Lectura de dataframes </b>

Para el análisis de sentimientos se utiliza la libreria nltk y descargar los paquetes correspondientes.

In [1]:
import pandas as pd 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pdb

In [2]:
df_reviews = pd.read_parquet('..\\datasets\\New_datasets\\user_reviews.parquet')

In [3]:
df_reviews.head(5)

,user_id,item_id,recommend,review,year_posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013


In [4]:
# Codigo de analisis de sentimiento de reviews, clasificando en '0' si es malo, '1' si es neutral y '2' si es positivo
analyzer = SentimentIntensityAnalyzer()
datos = []
def sentimental_analisis(df_reviews):     
    try:
        # Itera en cada linea de todos los reviews
        for review in df_reviews.review:            
            try:
                # Analisis del sentimiento y clasificacion numerica 1,2 o 3
                if review:            
                    score = analyzer.polarity_scores(review)
                    score.pop('compound') # Se elimina la columna 'compound' de score, no aporta a escoger si es negativa, neutra o positiva la reseña
                    match max(score, key = score.get): # Se obtiene el maximo y guarda clasificacion numerica en datos                                                
                            case 'neg':
                                datos.append(0)                 
                            case 'neu':
                                datos.append(1)       
                            case 'pos':
                                datos.append(2)
                            case _:
                                # En caso de algun error se espera input de usuario
                                pdb.set_trace()   
                else:
                    datos.append(1)

            except:
                    datos.append(1)
    except UnicodeDecodeError as e :
        print(e)      

        
    df = pd.DataFrame(datos,columns=['sentiment_analysis'])
    df.head(100)
    return df

In [5]:
#sentiment_analysis = sentimental_analisis(df_reviews)
df_reviews['sentiment_analysis'] = sentimental_analisis(df_reviews)

In [6]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49186 entries, 0 to 49185
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   recommend           49186 non-null  bool  
 3   review              49186 non-null  object
 4   year_posted         49186 non-null  int64 
 5   sentiment_analysis  49186 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 1.9+ MB


In [7]:
df_reviews.head(5)

,user_id,item_id,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,1
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,js41637,251610,True,I know what you think when you see this title ...,2014,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [8]:
df_reviews.to_parquet('..\\datasets\\New_datasets\\user_reviews_sentiments.parquet',index=False)